In [11]:
import os
import file_parser
import pandas as pd
import cv2
import numpy as np
import time
import h5py
import json

import file_parser

## Create CSV file \[file_location, output\] 

In [12]:
dataset_path = "E:\MachineLearning\Datasets\dogs-vs-cats\train"
dataset_csv = 'data/data.csv'

if not os.path.exists(dataset_csv):
    if not os.path.exists('data'):
        os.makedirs('data')
    df = file_parser.get_all_files()
    file_parser.save_as_csv(df,dataset_csv)
    print("Created csv file")
else:
    print("CSV file already exists. Skipping...")

CSV file already exists. Skipping...


## Create train and test data 

In [13]:
x_train_file = 'data/train.h5'
x_test_file = 'data/test.h5'
image_size_x = 24;
image_size_y = 24;
train_size = 0.8;
if not os.path.exists(x_test_file):
    file_parser.split_and_save(dataset_csv, image_size_x, image_size_y, train_size, x_train_file, x_test_file)
    print("Dataset split and saved as train.h5 and test.h5")
else:
    print("Train and Test files already exists. Skipping...")

Train and Test files already exists. Skipping...


## CUSTOM FUNCTIONS

In [14]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

# derivative of sigmoid
def dsigmoid(z):
    return z*(1-z)

def relu(z):
    # max(0,z)
    return z*(z>0)

# derivative of relu
def drelu(z):
    return z>0

##  Load train and test data

In [15]:
with h5py.File('data/train.h5','r') as train:
    X_train = train.get('X')[()]
    y_train = train.get('y')[()]

In [16]:
print(X_train.shape, y_train.shape)

(1728, 20000) (1, 20000)


In [17]:
with h5py.File('data/test.h5','r') as test:
    X_test = test.get('X')[()]
    y_test = test.get('y')[()]

In [18]:
print(X_test.shape, y_test.shape)

(1728, 5000) (1, 5000)


In [19]:
# Normalize the data

X_train = X_train/255
X_test = X_test/255

# Neural Network with 1 hidden layer

In [64]:
def forward_propagation(X, y , cache):
    W1 = cache['W1']
    b1 = cache['b1']
    W2 = cache['W2']
    b2 = cache['b2']

    Z1 = np.dot(W1, X)+b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1)+b2
    A2 = sigmoid(Z2)
    
    return {
        "Z1":Z1,
        "A1":A1,
        "Z2":Z2,
        "A2":A2
    }

In [97]:
def backward_propagation(X, y, W, cache):
    Z1 = cache['Z1']
    A1 = cache['A1']
    Z2 = cache['Z2']
    A2 = cache['A2']

    dZ2 = A2-y
    dW2 = np.sum(np.dot(dZ2, A1.T), axis=1, keepdims=True)/m
    db2 = np.sum(dZ2, axis=1, keepdims=True)/m

    dZ1 = np.dot(W.T, dZ2)*drelu(Z1)
    dW1 = np.sum(np.dot(dZ1, X.T), axis=1, keepdims=True)/m
    db1 = np.sum(dZ1, axis=1, keepdims=True)/m

    return {
        "dW2": dW2,
        "db2": db2,
        "dW1": dW1,
        "db1": db1
    }

In [98]:
def update_params(Wcache, cache, learning_rate):
    W1 = Wcache["W1"]
    W2 = Wcache["W2"]
    b1 = Wcache["b1"]
    b2 = Wcache["b2"]

    dW1 = cache["dW1"]
    dW2 = cache["dW2"]
    db1 = cache["db1"]
    db2 = cache["db2"]


    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1
    W2 = W2 - learning_rate*dW2
    b2 = b2 - learning_rate*db2
    
    return {
        "W1": W1,
        "b1": b1,
        "W2": W2,
        "b2": b2
    }

In [105]:
def run_nn(X, y, Wcache, learning_rate=0.1, stop_criteria=None, num_iterations=5000, print_cost=False):
    m = X.shape[0]
    _run_time=0
    error = stop_criteria+1 or 1
    i=0
    J = None

    while i<num_iterations and abs(error)>stop_criteria:
        i+=1
        t1=time.time()

        cache = forward_propagation(X, y, Wcache)
        diffcache = backward_propagation(X, y, Wcache["W2"], cache)
        Wcache = update_params(Wcache, diffcache, learning_rate)

        a = cache["A2"]>0.5
        J_prev = J or 0
        J = np.sum(y!=a)/m
        # J = -np.sum(y*np.log(a)+(1-y)*np.log(1-a))/m
        error = abs(J-J_prev)
        
        t2=time.time()
        _run_time+=(t2-t1)
        if print_cost:
            print("Iteration {i}: cost = {cost}, execution time: {time:.3f}s, error: {err:.5f}".format(i=i, cost=J, time=t2-t1, err=error))
    
    
    print("\nAfter {n} iterations: cost = {cost}, total execution time: {time:.3f}s".format(n=i, cost=J, time=_run_time))
    print(X.shape)
    print(len(a[0]))
    print(np.sum(y==a)/X.shape[1]*100)
    return {
        "w": Wcache['W2'].tolist(),
        "b": Wcache['b2'],
        "learning_rate": learning_rate,
        "cost": J
    }

Implementing NN with 1 hidden layer with 4 nodes and output layer with 1 node.

Dimensions of parameters:
X = (nx, m)  # m-> no. of examples, nx->size of array representing 1 image

W1 = dW1 = (4, nx) 

b1 = db1 = (4,1)

Z1 = A1 = dZ1 = dA1 = (4,m)

W2 = dW2 = (1,4)

b2 = db2 = (1,1)

Z2 = A2 = dZ2 = dA2 = (1,m)

In [106]:
m=X_train.shape[0]
learning_rate = 0.01
W1 = np.random.randn(4,m)*0.001
b1 = np.zeros((4,1))
W2 = np.random.randn(1,4)*0.001
b2 = np.zeros((1,1))
wcache = {
    "W1":W1,
    "b1":b1,
    "W2":W2,
    "b2":b2
}
n=100
show_epoch_details=True
stop_criteria=0.0001

res = run_nn(X_train, y_train, wcache, learning_rate, stop_criteria=stop_criteria, num_iterations=n,print_cost=show_epoch_details)

Iteration 1: cost = 5.782986111111111, execution time: 0.092s, error: 5.78299
Iteration 2: cost = 5.776041666666667, execution time: 0.066s, error: 0.00694
Iteration 3: cost = 5.776041666666667, execution time: 0.071s, error: 0.00000

After 3 iterations: cost = 5.776041666666667, total execution time: 0.228s
(1728, 20000)
20000
50.095


In [ ]:
np.sum(np.abs(y-a))

# ROUGH

In [1]:
import numpy as np

In [50]:
a = np.array([[1,-2,3]])
a.resize(3,1)
a

array([[ 1],
       [-2],
       [ 3]])

In [54]:
a = np.array([[1,2,3]])
b = np.array([[2,2,-1]])

In [55]:
a==b

array([[False,  True, False]])